အောက်ပါ notebook များကို run လုပ်ရန်အတွက်၊ မလုပ်ရသေးပါက `text-embedding-ada-002` ကို base model အဖြစ် အသုံးပြုသော model တစ်ခုကို deploy လုပ်ရန် လိုအပ်ပြီး၊ deployment name ကို .env ဖိုင်အတွင်း `AZURE_OPENAI_EMBEDDINGS_ENDPOINT` အဖြစ် သတ်မှတ်ပေးရန် လိုအပ်ပါသည်။


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

နောက်တစ်ခုမှာတော့ Embedding Index ကို Pandas Dataframe ထဲသို့ load ပြုလုပ်သွားပါမယ်။ Embedding Index ကို `embedding_index_3m.json` လို့ခေါ်တဲ့ JSON ဖိုင်ထဲမှာ သိမ်းဆည်းထားပါတယ်။ Embedding Index ထဲမှာ YouTube transcript တစ်ခုချင်းစီအတွက် Embeddings တွေကို 2023 အောက်တိုဘာလ နောက်ပိုင်းအထိ ပါဝင်ပါတယ်။


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

နောက်တစ်ခါမှာတော့ `get_videos` လို့ခေါ်တဲ့ function တစ်ခုကို ဖန်တီးသွားမှာပါ။ ဒီ function ကတော့ Embedding Index ထဲမှာ query ကို ရှာဖွေသွားမှာဖြစ်ပြီး၊ query နဲ့ အနီးစပ်ဆုံး ဗီဒီယို ၅ ခုကို ပြန်ပေးပါလိမ့်မယ်။ ဒီ function ရဲ့ လုပ်ဆောင်ပုံကတော့ အောက်ပါအတိုင်း ဖြစ်ပါတယ်-

1. အရင်ဆုံး Embedding Index ကို မိတ္တူတစ်စုံ ပြုလုပ်ပါတယ်။
2. နောက်တစ်ဆင့်မှာတော့ OpenAI Embedding API ကို အသုံးပြုပြီး query အတွက် Embedding ကိုတွက်ချက်ပါတယ်။
3. ပြီးရင်တော့ Embedding Index ထဲမှာ `similarity` လို့ခေါ်တဲ့ column အသစ်တစ်ခု ဖန်တီးပါတယ်။ ဒီ `similarity` column ထဲမှာတော့ query ရဲ့ Embedding နဲ့ ဗီဒီယို segment တစ်ခုချင်းစီရဲ့ Embedding တို့အကြား cosine similarity တန်ဖိုးတွေ ပါဝင်ပါတယ်။
4. နောက်တစ်ဆင့်မှာတော့ Embedding Index ကို `similarity` column နဲ့ စစ်ထုတ်ပါတယ်။ Cosine similarity တန်ဖိုး 0.75 ထက် များသော်လည်းကောင်း၊ ညီမျှသော်လည်းကောင်းရှိတဲ့ ဗီဒီယိုတွေကိုသာ ထည့်သွင်းစစ်ထုတ်ပါတယ်။
5. နောက်ဆုံးမှာတော့ Embedding Index ကို `similarity` column နဲ့ အလိုက်စီပြီး အနီးစပ်ဆုံး ဗီဒီယို ၅ ခုကို ပြန်ပေးပါတယ်။


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

ဒီဖန်ရှင်က အရမ်းလွယ်ပါတယ်၊ ဒါက ရှာဖွေမှုအတွက်ရလဒ်တွေကိုသာ ပုံနှိပ်ပြသပေးတာပါ။


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

၁။ အရင်ဆုံး Embedding Index ကို Pandas Dataframe ထဲသို့ တင်သွင်းပါမယ်။
၂။ နောက်တစ်ဆင့်မှာ အသုံးပြုသူကို query တစ်ခု ရိုက်ထည့်ဖို့ တောင်းဆိုပါမယ်။
၃။ ပြီးလျှင် `get_videos` function ကို ခေါ်ပြီး Embedding Index ထဲက query ကို ရှာဖွေပါမယ်။
၄။ နောက်ဆုံးမှာ `display_results` function ကို ခေါ်ပြီး ရလဒ်တွေကို အသုံးပြုသူကို ပြပါမယ်။
၅။ အသုံးပြုသူကို နောက်ထပ် query တစ်ခု ထပ်မံရိုက်ထည့်ဖို့ တောင်းဆိုပါမယ်။ ဒီလုပ်ငန်းစဉ်က အသုံးပြုသူက `exit` လို့ ရိုက်ထည့်မချင်း ဆက်လက်လုပ်ဆောင်သွားပါမယ်။

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.my.png)

သင့်ကို query တစ်ခု ရိုက်ထည့်ဖို့ တောင်းဆိုပါလိမ့်မယ်။ Query ကို ရိုက်ထည့်ပြီး enter နှိပ်ပါ။ Application က သက်ဆိုင်ရာ video စာရင်းကို ပြန်ပေးပါလိမ့်မယ်။ ထို့အပြင် မေးခွန်းအဖြေ ပါဝင်တဲ့ video မှာ တည်နေရာ link ကိုလည်း ပြန်ပေးပါမယ်။

စမ်းသပ်ကြည့်နိုင်တဲ့ query များမှာ -

- Azure Machine Learning ဆိုတာဘာလဲ?
- Convolutional neural networks ဘယ်လိုအလုပ်လုပ်သလဲ?
- Neural network ဆိုတာဘာလဲ?
- Jupyter Notebooks ကို Azure Machine Learning နဲ့ အသုံးပြုနိုင်မလား?
- ONNX ဆိုတာဘာလဲ?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**သတိပြုရန်**:
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှန်ကန်မှုအတွက် ကြိုးစားနေသော်လည်း၊ အလိုအလျောက်ဘာသာပြန်ခြင်းတွင် အမှားများ သို့မဟုတ် မမှန်ကန်မှုများ ပါဝင်နိုင်သည်ကို သတိပြုပါ။ မူရင်းစာရွက်စာတမ်းကို မူလဘာသာဖြင့်သာ အာဏာတရားရှိသော အရင်းအမြစ်အဖြစ် ယူဆသင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူ့ဘာသာပြန်ပညာရှင်များ၏ ဝန်ဆောင်မှုကို အသုံးပြုရန် အကြံပြုပါသည်။ ဤဘာသာပြန်ချက်ကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော နားလည်မှုမှားခြင်း သို့မဟုတ် အနားလည်အမှားများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
